In [1]:
from xclib.data import data_utils
import pandas as pd
from numpy import *
import numpy as np
import statistics

In [2]:
def read_labels(f_name):
    f = pd.read_csv(f_name, header = None,  encoding='ISO-8859-1') 
    f = f.to_numpy() 
    return f

In [3]:
Y_test = read_labels('/home/shreya/Sem6/COL774/A3/virus/ass3_parta_data/test_y.txt')
Y_train = read_labels('/home/shreya/Sem6/COL774/A3/virus/ass3_parta_data/train_y.txt')

In [4]:
X_test = data_utils.read_sparse_file('/home/shreya/Sem6/COL774/A3/virus/ass3_parta_data/test_x.txt', force_header=True)
X_train = data_utils.read_sparse_file('/home/shreya/Sem6/COL774/A3/virus/ass3_parta_data/train_x.txt', force_header=True)

/home/shreya/.local/lib/python3.6/site-packages/xclib-0.96-py3.6-linux-x86_64.egg/xclib/data/data_utils.py:173: UserWarning: Header mis-match from inferred shape!
  warnings.warn("Header mis-match from inferred shape!")


In [5]:
f_size = X_train.shape[1]
print(f_size)

482


In [6]:
# x = X_train[0].toarray()
# print((x[0][1]))
# print(X_train[8])

In [7]:
# l = list(range(0,X_test.shape[0]))
# print(feature_median(15,l, X_test))

In [8]:
#to get H_y
p_y0 = 0.0
p_y1 = 0.0
for label in Y_train:
    if(label==0):
        p_y0 += 1.0

p_y1 = len(Y_train) - p_y0
print("p_y0 = ", p_y0)
print("p_y1 = ", p_y1)
print(len(Y_train))
p_y0 = p_y0/len(Y_train)
p_y1 = p_y1/len(Y_train)


H_y = (-1)*p_y0*(np.log(p_y0)/np.log(2))
H_y = H_y - p_y1*(np.log(p_y1)/np.log(2))
print('H of y = ', H_y)

p_y0 =  30560.0
p_y1 =  34153.0
64713
H of y =  0.9977751575509852


In [9]:
def feature_median(f_index, i_list, X):
    med_list = []
    for i in i_list:
        temp = X[i].toarray()
        med_list.append(temp[0][f_index])
#     print(max(med_list))
    return median(med_list)

In [10]:
def H_entropy(i_list, X, Y):
    p0 = 1.0
    p1 = 1.0
    for i in i_list:
        if(Y[i]==1):
            p1 += 1.0
        else:
            p0 += 1.0
    p0 = p0/(len(i_list)+2)
    p1 = p1/(len(i_list)+2)    
    
#     print("p0 ", p0)
#     print("p1 ", p1)
    
    H = (p0*(np.log(1/p0)/np.log(2))) + (p1*(np.log(1/p1)/np.log(2))) 
    print(H)
    return H

In [11]:
# f_med = feature_median(list(range(0,len(Y_train))), X_train)
# print(f_med)
# print(H_entropy(1, f_med, list(range(0,len(Y_train))), X_train))

In [12]:
def mutual_info(f_index, i_list, X, Y):
    
    f_median = feature_median(f_index, i_list, X)
#     median1 = feature_median(f_index, i1_list, X)
    if f_median==0:
        return 1000000, [], []
    f0_list = []
    f1_list = []
    
    for i in i_list:
        temp = X[i].toarray()
        if(temp[0][f_index]<=f_median):
            f0_list.append(i)
        else:
            f1_list.append(i)
    
    print('med_feature: ', f_median)
#     print('med1: ',median1)
    
    H_y_x0 = H_entropy(f0_list, X, Y)
    H_y_x1 = H_entropy(f1_list, X, Y)    
    
    print('H0: ',H_y_x0)
    print('H1: ',H_y_x1)
    f0 = float(len(f0_list)+1)/(len(i_list)+1)
    f1 = float(len(f1_list)+1)/(len(i_list)+1)
    
    print('f0: ',f0)
    print('f1: ',f1)
    
    H_y_x = ((f0*H_y_x0) + (f1*H_y_x1))
    print('Entropy of y given x is = ', H_y_x)
    return H_y_x, f0_list, f1_list

In [13]:
#find best feature
def best_feature(i_list, X, Y):
    best_f = 0
    min_entropy = 1000000
    f0_best = []
    f1_best = []
    
    i0_list = []
    i1_list = []
    
    #to calculate H(y)
    for i in i_list:
        if(Y[i]==0):
            i0_list.append(i)
        else:
            i1_list.append(i)
            
    p_y0 = float(len(i0_list))/len(Y)
    p_y1 = float(len(i1_list))/len(Y)
    
    H_y = (p_y0*(np.log(1/p_y0)/np.log(2))) + (p_y1*(np.log(1/p_y1)/np.log(2)))  
    
    for i in range(f_size):
        val, f0, f1 = mutual_info(i, i_list, X, Y)
        if (min_entropy>val):
            min_entropy = val
            best_f = i
            f0_best = f0
            f1_best = f1
    
    median_val = feature_median(best_f, i_list, X)
    return best_f, median_val, f0_list, f1_list

In [14]:
# print(best_feature(list(range(0,len(Y_train))), X_train, Y_train))

In [15]:
class Node:
    def __init__(self, feature, median, height):
        self.left = None
        self.right = None
        self.feature = feature
        self.median = median
        self.height = height
    
    def insert_left(self, node):
        if self.left is None:
            self.left = node
    
    def insert_right(self, node):
        if self.right is None:
            self.right = node  

In [16]:
def make_tree(i_list, X, Y, height):
    if(height==0):
        case0 = 0
        case1 = 0
        for i in i_list:
            if Y[i]==0:
                case0 += 1
            else:
                case1 += 1
        if case0 > case1:
            return 0
        else:
            return 1
    else:
        feature, f_median, f0_list, f1_list = best_feature(i_list, X, Y)
        f_node = Node(feature, f_median, height)
        print("At height: ", height, " feature: ", feature, " f_median: ", f_median)
        left_node = make_tree(f0_list, X, Y, height-1)
        right_node = make_tree(f1_list, X, Y, height-1)
        f_node.insert_left(left_node)
        f_node.insert_right(right_node)        
        return f_node

In [17]:
#define decision tree:
height = 2
i_list = range(0, len(Y_train))

Tree = make_tree(i_list, X_train, Y_train, height)

med_feature:  1.0
0.9804962905501919
0.9936458870760403
H0:  0.9804962905501919
H1:  0.9936458870760403
f0:  0.5790555366690361
f1:  0.42095991593781873
Entropy of y given x is =  0.9860468948220298
med_feature:  9.0
0.9460983644738901
0.978261560664015
H0:  0.9460983644738901
H1:  0.978261560664015
f0:  0.5141236826652656
f1:  0.48589176994158917
Entropy of y given x is =  0.9617408164837606
med_feature:  2.0
0.9792992304860615
0.9966358461498324
H0:  0.9792992304860615
H1:  0.9966358461498324
f0:  0.5217263652378156
f1:  0.47828908736903913
Entropy of y given x is =  0.9876062772959561
med_feature:  8.0
0.9743339819011949
0.9956647321436192
H0:  0.9743339819011949
H1:  0.9956647321436192
f0:  0.5008653459838674
f1:  0.4991501066229873
Entropy of y given x is =  0.9849962841590167
med_feature:  4.0
0.9992757121850022
0.9954275757208431
H0:  0.9992757121850022
H1:  0.9954275757208431
f0:  0.5023178910282164
f1:  0.4976975615786383
Entropy of y given x is =  0.9973759454648883
med_featu

KeyboardInterrupt: 

In [ ]:
predict